<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import sqlalchemy as sa


In [4]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [5]:
# предлагаемый в курсе способ подключение к БД выдаёт предупреждение о необходимости подключения с помощью интсрументов
# библиотеки SQLAlchemy. Это связано с политекой разработчиков pandas, поэтому курс в этой части вероятно требует
# переработки
connection = f'postgresql+psycopg2://{USER}:{PASSWORD}@{HOST}:{PORT}/{DBNAME}'

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [51]:
query_3_1 = f'''select
                       count(*) "Количество вакансий"
                  from vacancies
'''

In [80]:
# результат запроса
df_3_1 = pd.read_sql_query(query_3_1, connection)
df_3_1.style.hide(axis="index") # прячем столбец индексов 


Количество вакансий
49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [53]:
# текст запроса
query_3_2 = f'''select
                       count(*) "Количество работодателей"
                  from employers
'''

In [81]:
# результат запроса
df_3_2 = pd.read_sql_query(query_3_2, connection)
df_3_2.style.hide(axis="index") # прячем столбец индексов 

Количество работодателей
23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [55]:
# текст запроса
query_3_3 = f'''select
                       count(*) "Количество регионов"
                  from areas
'''

In [82]:
# результат запроса
df_3_3 = pd.read_sql_query(query_3_3, connection)
df_3_3.style.hide(axis="index") # прячем столбец индексов 

Количество регионов
1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [57]:
# текст запроса
query_3_4 = f'''select
                       count(*) "Количество сфер деятельности"
                  from industries
'''

In [83]:
# результат запроса
df_3_4 = pd.read_sql_query(query_3_4, connection)
df_3_4.style.hide(axis="index") # прячем столбец индексов 

Количество сфер деятельности
294


***

### выводы по предварительному анализу данных

Выборка содержит около 50 тысяч вакансий в почти 300 сферах деятельности от более 23 тысяч работодателей в 1362 городах 

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [85]:
# текст запроса
query_4_1 = f'''select
                       count(v.id) cnt,
                       a.name
                  from vacancies v
                  join areas a
                    on a.id = v.area_id
              group by a.id
              order by 1 desc
                 limit 5
'''

In [86]:
# результат запроса
df_4_1 = pd.read_sql_query(query_4_1, connection)
df_4_1.style.hide(axis="index") # прячем столбец индексов 

cnt,name
5333,Москва
2851,Санкт-Петербург
2112,Минск
2006,Новосибирск
1892,Алматы


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [36]:
# текст запроса
query_4_2 = f'''select
                       count(*) "Количество вакансий с зарплатой"
                  from (
                select
                       salary_from,
                       salary_to
                  from vacancies
                 where salary_from is not null
                    or salary_to is not null
                       ) v
'''

In [87]:
# результат запроса
df_4_2 = pd.read_sql_query(query_4_2, connection)
df_4_2.style.hide(axis="index") # прячем столбец индексов 

Количество вакансий с зарплатой
24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [59]:
# текст запроса
query_4_3 = f'''select
                       round(avg(salary_from)) "Среденее нижнее",
                       round(avg(salary_to)) "Среднее верхнее"
                  from vacancies v
           
'''

In [88]:
# результат запроса
df_4_3 = pd.read_sql_query(query_4_3, connection)
df_4_3.style.hide(axis="index") # прячем столбец индексов 

Среденее нижнее,Среднее верхнее
71065.000000,110537.000000


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [61]:
# текст запроса
query_4_4 = f'''select
                       schedule || '-' || employment comb,
                       count(id)
                  from vacancies v
              group by comb
              order by 2 desc
                offset 1
                 limit 1
'''           

In [89]:
# результат запроса
df_4_4 = pd.read_sql_query(query_4_4, connection)
df_4_4.style.hide(axis="index") # прячем столбец индексов 

comb,count
Удаленная работа-Полная занятость,7802


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [91]:
# текст запроса
query_4_5 = f'''select
                       experience,
                       count(id)
                  from vacancies v
              group by experience
                having experience is not null
              order by 2
    
'''           

In [92]:
# результат запроса
df_4_5 = pd.read_sql_query(query_4_5, connection)
df_4_5.style.hide(axis="index") # прячем столбец индексов 

experience,count
Более 6 лет,1337
Нет опыта,7197
От 3 до 6 лет,14511
От 1 года до 3 лет,26152


***

In [48]:
# выводы по детальному анализу вакансий

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [93]:
# текст запроса
query_5_1 = f'''select
                       e.name,
                       count(v.id)
                  from vacancies v
                  join employers e
                    on e.id = v.employer_id
              group by e.name
              order by 2 desc
                 limit 5
    
'''       

In [94]:
# результат запроса
df_5_1 = pd.read_sql_query(query_5_1, connection)
df_5_1.style.hide(axis="index") # прячем столбец индексов 

name,count
Яндекс,1933
Ростелеком,491
Тинькофф,444
СБЕР,428
Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [95]:
# текст запроса
query_5_2 = f'''select
                       a.name
                  from areas a
             left join vacancies v
                    on a.id = v.area_id
                  join employers e
                    on a.id =  e.area
              group by a.name
                having count(v.id) = 0
              order by count(e.id) desc
                 limit 1
    
'''       

In [96]:
# результат запроса
df_5_2 = pd.read_sql_query(query_5_2, connection)
df_5_2.style.hide(axis="index") # прячем столбец индексов 

name
Россия


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [75]:
# текст запроса
query_5_3 = f'''select
                       e.name,
                       count(distinct v.area_id) cnt
                  from vacancies v
                  join employers e
                    on e.id = v.employer_id
              group by e.id
              order by cnt desc
                 limit 5
                
'''       

In [79]:
# результат запроса
df_5_3 = pd.read_sql_query(query_5_3, connection)
df_5_3.style.hide(axis="index")

name,cnt
Яндекс,181
Ростелеком,152
Спецремонт,116
Поляков Денис Иванович,88
ООО ЕФИН,71


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [63]:
# текст запроса
query_5_4 = f'''select
                       count(e.id) "Количество работодателей без сферы"
                  from employers e
             left join employers_industries ei
                    on e.id = ei.employer_id
                 where ei.employer_id is null
'''       

In [97]:
# результат запроса
df_5_4 = pd.read_sql_query(query_5_4, connection)
df_5_4.style.hide(axis="index") # прячем столбец индексов 

Количество работодателей без сфер
8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [98]:
# текст запроса
query_5_5 = f'''select
                       e.name
                  from employers e
                  join employers_industries ei
                    on e.id = ei.employer_id
              group by e.name
                having count(ei.industry_id) = 4
              order by e.name
                offset 2
                 limit 1
'''       

In [99]:
# результат запроса
df_5_5 = pd.read_sql_query(query_5_5, connection)
df_5_5.style.hide(axis="index") # прячем столбец индексов 

name
2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [101]:
# текст запроса
query_5_6 = f'''select
                       count(e.name)
                  from employers e
                  join employers_industries ei
                    on e.id = ei.employer_id
                  join industries i
                    on i.id = ei.industry_id
                 where i.name = 'Разработка программного обеспечения'
'''

In [102]:
# результат запроса
df_5_6 = pd.read_sql_query(query_5_6, connection)
df_5_6.style.hide(axis="index") # прячем столбец индексов 

count
3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%%D0%93%%D0%%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [19]:
# код для получения списка городов-милионников
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8#%D0%A1%D0%BF%D0%B8%D1%81%D0%BE%D0%BA_%D0%B3%D0%BE%D1%80%D0%BE%D0%B4%D0%BE%D0%B2-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D0%BE%D0%B2'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
links = page.find_all('a')
cities_bill = [link.text for link in links[96:112]]
cities_bill = tuple(cities_bill)
cities_bill

('Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Красноярск',
 'Екатеринбург',
 'Нижний Новгород',
 'Самара',
 'Казань',
 'Пермь',
 'Омск',
 'Челябинск',
 'Ростов-на-Дону',
 'Волгоград',
 'Уфа',
 'Воронеж',
 'Краснодар')

In [16]:
# текст запроса
query_5_7 = f'''select
                       a.name,
                       count(v.id) cnt
                  from areas a
                  join vacancies v
                    on a.id = v.area_id
                  join employers e
                    on e.id = v.employer_id
                 where e.name = 'Яндекс'
                   and a.name in {cities_bill}
                 group by a.name
                 union
                select
                       'total',
                       count(v.id)
                  from areas a
                  join vacancies v
                    on a.id = v.area_id
                  join employers e
                    on e.id = v.employer_id
                 where e.name = 'Яндекс'
                   and a.name in {cities_bill}
              order by cnt    
'''

In [17]:
# результат запроса
df_5_7 = pd.read_sql_query(query_5_7, connection)
df_5_7.style.hide(axis="index") # прячем столбец индексов 

name,cnt
Омск,21
Челябинск,22
Красноярск,23
Волгоград,24
Пермь,25
Казань,25
Ростов-на-Дону,25
Уфа,26
Самара,26
Краснодар,30


***

In [66]:
# выводы по анализу работодателей

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [112]:
# текст запроса
query_6_1 = f'''select
                       count(id) "Количество вакансий"
                  from vacancies v
                 where lower(name)
            similar to '%(data|данн)%'
'''

In [113]:
# результат запроса
df_6_1 = pd.read_sql_query(sa.text(query_6_1), connection)
df_6_1.style.hide(axis="index") # прячем столбец индексов 

Количество вакансий
1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [114]:
ds_keyword = "'_*(data scientist|data science|исследователь данных|ml|machine learning|машинн%обучен%)_*'"
query_6_2 = f'''select
                       count(id)
                  from vacancies v
                 where replace(lower(name), 'html', '')
            similar to {ds_keyword}
                   and (lower(name) like '%junior%'
                    or experience like 'Нет опыта'
                    or employment like 'Стажировка')          
'''

In [115]:
# результат запроса
df_6_2 = pd.read_sql_query(sa.text(query_6_2), connection)
df_6_2.style.hide(axis="index") # прячем столбец индексов 

count
51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

In [116]:
# текст запроса
query_6_3 = f'''select
                       count(id)
                  from vacancies v
                 where (replace(lower(name), 'html', '')
            similar to {ds_keyword})
                   and (lower(key_skills)
            similar to '%(sql|postgres)%')           
'''

In [117]:
# результат запроса
df_6_3 = pd.read_sql_query(sa.text(query_6_3), connection)
df_6_3.style.hide(axis="index") # прячем столбец индексов 

count
229


4. Проверьте, насколько популярен Python в требованиях работодателей к DS. Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [118]:
# текст запроса
query_6_4 = f'''select
                       count(id)
                  from vacancies v
                 where (replace(lower(name), 'html', '')
            similar to {ds_keyword})
                   and (lower(key_skills)
            similar to '%(python)%')
'''

In [119]:
# результат запроса
df_6_4 = pd.read_sql_query(sa.text(query_6_4), connection)
df_6_4.style.hide(axis="index") # прячем столбец индексов 

count
357


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [126]:
# текст запроса
query_6_5 = f'''select
                       round(avg(length(key_skills) - length(replace(key_skills, chr(9), ''))+1), 2)
                  from vacancies v
                 where (replace(lower(name), 'html', '')
            similar to {ds_keyword})
'''

In [129]:
# результат запроса
df_6_5 = pd.read_sql_query(sa.text(query_6_5), connection)
df_6_5.style.hide(axis="index") # прячем столбец индексов 

round
6.550000


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [142]:
# текст запроса
query_6_6 = f'''select
                       round(avg(coalesce((salary_from+salary_to)/2, salary_from, salary_to)), 0) "Средняя зарплата"
                  from vacancies v
                 where (replace(lower(name), 'html', '')
            similar to {ds_keyword})
                   and (salary_from is not null or salary_to is not null)
              group by experience
                having experience = 'От 3 до 6 лет'
'''

In [143]:
# результат запроса
df_6_6 = pd.read_sql_query(sa.text(query_6_6), connection)
df_6_6.style.hide(axis="index") # прячем столбец индексов 

Средняя зарплата
256454.000000


***

**Выводы:**

1. Анализ ваканчий выявил весьма небольшое количество (480) предложений для Data Science, а для таких, у кого нет опыта работы вобще 51. Таким образом можно сделать вывод, что рынок перенасыщен "джунами", что является сигналом для обрахзовательных организаций для ввода программ повышения квалификации и, прежде всего, направленных на приобретение практического опыта, в том числе создание "песочниц" для реализации pet-проектов.
2. Небольшой спроса на неопытных датасайентистов подтверждается и анализом предлагаемой заработной платы - зарплата тех, кто отработатл по специальности более трёх лет, в 3 раза выше начинающего сотрудника.
3. Меньщше половины (201) вакансий Data Science предполагают знание специалистами SQL или postgres, что обясняется развитием универсальных инструментов прогамирования, например, Python, знание которго требуется в 351 вакансии. В среднем работодатели вобще хотят видеть у своих сотрудников больше 6 'hard-skills", что опять же является сигналов для школ и ВУЗов для расширения стэка выпускников, в тмо числе и в рамках дополнительной подготовки специалистов Data Science.
4. В ходе анализа выявлено также небольшое отличе между выборкой вакансий, которые действительно тносятся к Data Science и от выборки, используемой для подготовки ответов на платформе (из-за упрощённого фильтра в неё попали другие специальности). Это обстоятельство может привести к невозможности найти правильные ответы на задания, а также некоторому искажению выводов настоящего анализа, что продемонстрировано в результах запросов в данной разделе.

# Общий вывод по проекту

In [ ]:
# подведем итог исследования, обобщите выводы
# здесь можно (это будет плюсом) провести дополнительные исследования данных, сделать прогнозы, продумать варианты продолжения исследования